In [2]:
import os
import pandas as pd
import wget
import sys
import subprocess
from subprocess import call
import matplotlib as plt

import json

import numpy as np

from metamind.api import ClassificationData, ClassificationModel, set_api_key

sys.path.append(os.environ["HOME"])
from ds_config import *

print os.environ["HOME"]
MAIN_PATH = os.environ["HOME"]+"/GLaDOS_Workspace/"
#os.mkdir(MAIN_PATH)
#os.mkdir(MAIN_PATH+"Videos/")
#os.mkdir(MAIN_PATH+"Images/")

/Users/williamspearman


# FFMPEG Scrips

In [4]:
def getVideoInfo(file_name):
    bashCommand = "ffprobe -show_streams " + file_name
    process = subprocess.Popen(bashCommand.split(" "), stdout=subprocess.PIPE)
    output = process.communicate()[0].split("\n")
    ret_dict = { }
    for x in output:
        if "=" in x:
            key_vals = x.split("=")
            ret_dict[key_vals[0]] = key_vals[1]
    #print ret_dict
    return ret_dict


def getGetSignalBkgTriplets(file_name, snap_time, pm_secs=1.):
    ret_val = { }
    info_dict = getVideoInfo(file_name)
    required_vals = { "duration", "nb_frames", "avg_frame_rate", "sample_aspect_ratio" }
    if len(required_vals.intersection(set(info_dict.keys())))!=len(required_vals):
        return None
    frame_rate = 0.
    try:
        frame_rate = float(info_dict["avg_frame_rate"].split("/")[0])/float(info_dict["avg_frame_rate"].split("/")[1])
    except:
        return None
    num_frames = float(info_dict["nb_frames"])
    duration = float(info_dict["duration"])

    if int(num_frames/frame_rate-duration)!= 0:
        return None
    
    pm_frames = pm_secs * frame_rate
    snap_frame = snap_time * frame_rate
    
    snap_frame_low = snap_frame-pm_frames
    snap_frame_high = snap_frame+pm_frames
    if snap_frame_low<0:
        return None
    if snap_frame_high>num_frames:
        return None
    
    int1 = [ pm_frames, max(0,snap_frame_low-pm_frames) ]
    int2 = [ min(pm_frames+snap_frame_high, num_frames), num_frames-pm_frames ]
    bkg_frame = getRandInt(int1, int2)
    
    ret_val["SNAP"] = [ snap_frame_low/frame_rate, snap_frame/frame_rate, snap_frame_high/frame_rate ]
    ret_val["NOT_SNAP"] = [ (bkg_frame-pm_frames)/frame_rate, bkg_frame/frame_rate, (bkg_frame+pm_frames)/frame_rate ]
    
    return ret_val
    
def downloadUrl(url):
    base_name = ""
    try:
        base_name = url.split("/")[-1]
    except:
        return ""
    
    output_name = MAIN_PATH + "Videos/" + base_name
    if os.path.isfile(output_name)==False:
        wget.download(url, MAIN_PATH+"Videos/")
    
    if os.path.isfile(output_name)==True:
        return output_name
    else:
        return ""



def makeImageMontage(url, snap_time, time_offset=0.5):
    ret_val = [ ]
    video_path = downloadUrl(url)
    #print "BLah %f" % 0.2
    base_name = ""
    try:
        base_name = video_path.split("/")[-1].split(".")[0]
    except:
        print "Failed splitting: %s" % video_path
        return None
    input_times = getGetSignalBkgTriplets(video_path, snap_time, time_offset)
    if input_times==None:
        return ret_val
    #resolution = resolution.replace(":", "x")
    #name1 = "blah"
    #name2 = "blah"
    ffmpeg_cmds = { }
    file_names = { }
    for mykey in input_times.keys():
        output_name = MAIN_PATH + "Images/" + base_name
        ffmpeg_cmds[mykey] = [ ]
        file_names[mykey] = [ ]
        for x, i in zip(input_times[mykey], range(0,len(input_times[mykey]))):
            file_name = "%s_%i_%s.jpg" % (output_name, i, mykey)
            file_names[mykey].append(file_name)
            ffmpeg_cmds[mykey].append("ffmpeg -ss %f -i %s -vframes 1 %s 2> /dev/null" % (x, video_path, file_name))
        #montage_cmd = "montage -tile 1x3 -geometry 512x256+0+0 "
        montage_cmd = "montage -tile 1x3 -geometry 512x+0+0 "
        for thecmd, fname in zip(ffmpeg_cmds[mykey], file_names[mykey]):
            call(thecmd, shell=True)
            montage_cmd = montage_cmd+fname+" "
        montage_name = "%s_%i_%s_MONTAGE.jpg" % (output_name, int(1000*input_times[mykey][1]), mykey)
        montage_cmd = montage_cmd + " " + montage_name
        call(montage_cmd,shell=True)
        for fname in file_names[mykey]:
            os.remove(fname)
        ret_val.append((montage_name, mykey))
        
        
        
            
    return ret_val


def getSnapImage(url, snap_time, formation):
    #ret_val = [ ]
    video_path = downloadUrl(url)
    if video_path=="":
        return ( "", formation )
    #print "BLah %f" % 0.2
    base_name = ""
    try:
        base_name = video_path.split("/")[-1].split(".")[0]
    except:
        print "Failed splitting: %s" % video_path
        return ( "", formation )
    output_name = MAIN_PATH + "Images/" + base_name

    file_name = "%s_%s.jpg" % (output_name, formation)
    ffmpeg_cmd = "ffmpeg -ss %f -i %s -vframes 1 %s 2> /dev/null" % (snap_time, video_path, file_name)
    call(ffmpeg_cmd, shell=True)
    os.remove(video_path)
            
    return ( file_name, formation )


In [5]:
print getSnapImage("http://vf.hudl.com/9n6/81k/bab387d7-c875-4df6-89f7-3a0c910deb6a/lelyp36kuaykgm1j-3000_1100.mp4",\
                       8.809,  "TRIPS")

('/Users/williamspearman/GLaDOS_Workspace/Images/lelyp36kuaykgm1j-3000_1100_TRIPS.jpg', 'TRIPS')


In [7]:
formationRDD = sc.textFile("s3n://hudl-hadoop/GLaDOS/formation_data/").map(json.loads)


In [10]:
formationRDD.take(2)

[{u'formation': u'TRIPS',
  u'off form': u'TRIPS',
  u'play type': u'Run',
  u'video': {u'snap confidence': 1,
   u'snap time': 3.804,
   u'urls': [u'http://vi.hudl.com/ynt/bmm/6db9a53a-0f69-41fc-8ed3-6d905d5c9240/hexwejs1qxsweu5j-1100.mp4']}},
 {u'formation': u'TRIPS',
  u'off form': u'TRIPS OPEN',
  u'play type': u'Run',
  u'video': {u'snap confidence': 1,
   u'snap time': 11.478,
   u'urls': [u'http://va.hudl.com/emb/xjy/da56e2d0-3ad1-44b4-8654-578a7619d4fe/5cyll3esqcydlziq-1100.mp4']}}]

In [25]:
finalRDD = formationRDD.map(lambda x: ( x["video"]["urls"][-1], x["formation"], x["video"]["snap time"] ))
tripsRDD = finalRDD.filter(lambda x: x[1]=="TRIPS").cache()
iformationRDD = finalRDD.filter(lambda x: x[1]=="I FORMATION").cache()
emptyRDD = finalRDD.filter(lambda x: x[1]=="EMPTY").cache()

trips_size = tripsRDD.count()
i_size = iformationRDD.count()
empty_size = emptyRDD.count()
num_wanted = float(min(trips_size,min(i_size, empty_size)))

tripsRDD_small = tripsRDD.sample(False, num_wanted/trips_size)
iformationRDD_small = iformationRDD.sample(False, num_wanted/i_size)
emptyRDD_small = emptyRDD.sample(False, num_wanted/empty_size)

finalRDD = tripsRDD_small\
    .union(iformationRDD_small)\
    .union(emptyRDD_small)


In [26]:
print "Trips: %i" % tripsRDD_small.count()
print "Empty: %i" % emptyRDD_small.count()
print "I Formation: %i" % iformationRDD_small.count()

Trips: 1024
Empty: 990
I Formation: 984


In [29]:
finalRDD.take(1)

[(u'http://vj.hudl.com/krt/rlp/a5926d58-9ac5-4082-8d29-fe0923ba2f9f/088-ziyr_1100.mp4',
  u'TRIPS',
  6.139)]

In [30]:
test_inputRDD = finalRDD\
    .map(lambda x: getSnapImage(x[0],x[2],x[1]))\
    .cache()
    #.filter(lambda x: x != "")\
    
    
#

In [ ]:
training_inputs = test_inputRDD.collect()

In [ ]:
set_api_key(metamind_api_key)


In [ ]:
training_data = ClassificationData(private=True, data_type="image", name="formation_detection")




In [ ]:
training_data.add_samples(training_inputs, input_type="files")


In [ ]:
classifier = ClassificationModel(private=True, name="formation_detection")
classifier.fit_async(training_data)



In [9]:
plt.figure(figsize=(10, 5))
print classifier.stats()["time_left"] / 60.
#print clf02.stats()["time_left"] / 60.
#print clf03.stats()["time_left"] / 60.

plt.plot(classifier.stats()["training_stats"]["train_accuracy"])
#plt.plot(clf02.stats()["training_stats"]["train_accuracy"])
#plt.plot(clf03.stats()["training_stats"]["train_accuracy"])
plt.ylim([0, 1])
plt.show() 

AttributeError: 'module' object has no attribute 'figure'

In [88]:
preds = classifier.predict(fg_test_set, input_type="files")

/home/ubuntu/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/ubuntu/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
Traceback (most recent call last):
  File "/home/ubuntu/anaconda/lib/python2.7/site-packages/metamind/api/utils.py", line 34, in make_request
    return validate_result(response)
  File "/home/ubuntu/anac

In [94]:
preds = pd.DataFrame([
    classifier.predict(fg, input_type="files") for fg in fg_test_set
])

/home/ubuntu/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/ubuntu/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/ubuntu/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL 

In [102]:
pred_dicts = []

for d in preds.values:
    pred_dicts.append(d[0])

In [103]:
preds = pd.DataFrame(pred_dicts)

In [112]:
preds.groupby("label").label.count()

label
fg       153
punt      25
Name: label, dtype: int64